In [2]:
from HGCal_ImageAlgo3D import *
from utility_benchmark import *
from timeit import default_timer as timer

%matplotlib inline
plt.style.use('classic')
plt.rc("figure",facecolor="w",figsize=(6,4))
plt.rc("font",size=10)
plt.rc("savefig",dpi=300)

Welcome to JupyROOT 6.10/00


In [3]:
DatasetDir  = '/home/zchen/Documents/ImageAlogrithm3D/data/'
DatasetFile = 'CMSSW93X_pt35_pu000'
df = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_rechit.pkl")
dfgen = pd.read_pickle(DatasetDir+"input/"+DatasetFile+"_gen.pkl")

# 1. Define Fitness

In [7]:
def TranslateDNA(DNA):
    para_upper = np.array([2,8,1,10])
    para_lower = np.array([0,2,0,2])
    para = DNA*(para_upper-para_lower)+para_lower
    return para

def DefineFitness(DNA):
    para = TranslateDNA(DNA)
    imagalgo = ImagingAlgo(AFFINITY_Z = para[0],
                           KERNAL_R   = para[1],
                           KERNAL_EXPC= para[2],
                          )
    
    dfresultclus = imagalgo.RunImagingAlgo(df,N=100,verb=False)
    bm = Benchmark(DatasetFile, dfresultclus, N=100)
    sigma,mean = bm.getEffSigma_EnergyEfficiency(deltarho = para[3])
    
    fitness = 1/sigma
    return fitness

# 2. Define GA and Optimize

In [8]:
from HGCal_GeneticOptimizer import *
from IPython.display import clear_output

ga = GA(pop_size    = 10,
        DNA_size    = 4, 
        DNA_fitness = DefineFitness,
        cross_rate  = 0.8,
        mutate_rate = 0.01,
        noise_rate  = 0.4,
        noise_amp   = 0.02
       )

In [ ]:
fitness_list = []
best_para_list = []
for generation in range(50):
    print(generation)
    clear_output(wait=True)
    
    ga.evolve()
    fitness  = ga.get_pop_fitness()
    best_DNA = ga.pop[np.argmax(fitness)]
    fitness_list.append(fitness)
    best_para_list.append(TranslateDNA(best_DNA))

8
